In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
# Read data and labels
data = pd.read_csv("../data/kmeans_data/data.csv", header = None)
labels = pd.read_csv('../data/kmeans_data/label.csv', header = None)

data.shape

(10000, 784)

In [3]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Define class to implement kmeans
class KMeans():
    def __init__(self, k, distance, max_iter):
        self.k = k
        self.max_iter = max_iter
        if distance == "euclidean":
            self.distance_function = self.__sumOfSquares()
        elif distance == 'cosine':
            self.distance_function = self.__cosine()
        else:
            self.distance_function = self.__jaccard()

    def __euclidean(self, x, centroids):
        euclid = []
        for center in centroids:
            distances.append(np.linalg.norm(x - center))
        return euclid

    def __cosine(self, x, centroids):
        cosine = []
        for center in centroids:
            cosine.append(np.dot(x, center) / (np.linalg.norm(x) * np.linalg.norm(center)))
        return cosine

    def __jaccard(self, x, centroids):
        jaccard = []
        for center in centroids:
            temp_jaccard = np.sum(min(x, center)) / np.sum(max(x, center))
            jaccard.append(1 - temp_jaccard)
        return jaccard

    def fit(self, X_train):
        # Initially, randomly select k datapoints as centrod.
        centroids = []
        for i in range(self.k):
            random_row = X_train.sample()
            centroids.append(np.random.choice(random_row))
        
        # Iteratively repeat and update the centroids
        iterations = 0
        prev_centroids = None
        while(iterations < 300 and prev_centroids != centroids):
            # Save the previous centroids so that current ones can be overwritten
            prev_centroids = centroids

            # Increment the number of iterations
            iterations += 1

In [7]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
